# Gauge Chart - KPI and Progress Visualization

**Use Case**: Display single metrics against targets, progress indicators, dashboard KPIs, performance scorecards

This notebook demonstrates how to create effective gauge charts for visualizing key performance indicators and progress metrics.


In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from matplotlib.patches import Wedge, Circle, Rectangle
import matplotlib.patches as mpatches
from matplotlib.collections import PatchCollection
import warnings
warnings.filterwarnings('ignore')

# Set style for better-looking plots
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("Set2")

# Set random seed for reproducibility
np.random.seed(42)

print("Gauge chart visualization libraries loaded!")


In [ ]:
# Create sample datasets for gauge charts
# 1. Sales Performance Dashboard
sales_kpis = pd.DataFrame({
    'KPI': ['Revenue', 'New Customers', 'Customer Satisfaction', 'Market Share', 
            'Sales Team Performance', 'Lead Conversion Rate'],
    'Current_Value': [2.8, 450, 4.2, 15.2, 78, 12.5],
    'Target_Value': [3.0, 500, 4.5, 18.0, 85, 15.0],
    'Min_Value': [0, 0, 1, 0, 0, 0],
    'Max_Value': [4.0, 600, 5.0, 25.0, 100, 20.0],
    'Unit': ['$M', 'Count', 'Rating', '%', 'Score', '%'],
    'Status': ['Good', 'Warning', 'Good', 'Poor', 'Good', 'Warning']
})

# Calculate achievement percentages
sales_kpis['Achievement_Pct'] = (sales_kpis['Current_Value'] / sales_kpis['Target_Value'] * 100).round(1)
sales_kpis['Progress_Pct'] = ((sales_kpis['Current_Value'] - sales_kpis['Min_Value']) / 
                             (sales_kpis['Max_Value'] - sales_kpis['Min_Value']) * 100).round(1)

# 2. Manufacturing Operations Dashboard
manufacturing_kpis = pd.DataFrame({
    'KPI': ['Overall Equipment Effectiveness', 'Quality Score', 'Safety Incidents', 
            'Production Volume', 'Cost per Unit', 'On-time Delivery'],
    'Current_Value': [82.5, 96.2, 2, 8500, 12.80, 94.5],
    'Target_Value': [85.0, 98.0, 0, 9000, 12.00, 95.0],
    'Min_Value': [0, 0, 0, 0, 8.00, 0],
    'Max_Value': [100, 100, 10, 10000, 20.00, 100],
    'Unit': ['%', '%', 'Count', 'Units', '$', '%'],
    'Status': ['Warning', 'Good', 'Warning', 'Good', 'Warning', 'Good']
})

manufacturing_kpis['Achievement_Pct'] = (manufacturing_kpis['Current_Value'] / manufacturing_kpis['Target_Value'] * 100).round(1)
manufacturing_kpis['Progress_Pct'] = ((manufacturing_kpis['Current_Value'] - manufacturing_kpis['Min_Value']) / 
                                     (manufacturing_kpis['Max_Value'] - manufacturing_kpis['Min_Value']) * 100).round(1)

# Special handling for safety incidents (lower is better)
safety_idx = manufacturing_kpis[manufacturing_kpis['KPI'] == 'Safety Incidents'].index[0]
manufacturing_kpis.loc[safety_idx, 'Achievement_Pct'] = (
    (manufacturing_kpis.loc[safety_idx, 'Target_Value'] / 
     max(manufacturing_kpis.loc[safety_idx, 'Current_Value'], 0.1)) * 100).round(1)
manufacturing_kpis.loc[safety_idx, 'Progress_Pct'] = (
    (1 - manufacturing_kpis.loc[safety_idx, 'Current_Value'] / manufacturing_kpis.loc[safety_idx, 'Max_Value']) * 100).round(1)

# 3. IT System Performance Dashboard
it_kpis = pd.DataFrame({
    'KPI': ['System Uptime', 'Response Time', 'CPU Utilization', 'Memory Usage', 
            'Network Latency', 'Security Score'],
    'Current_Value': [99.2, 150, 68, 74, 25, 87],
    'Target_Value': [99.9, 100, 70, 80, 20, 90],
    'Min_Value': [95.0, 50, 0, 0, 0, 0],
    'Max_Value': [100, 500, 100, 100, 100, 100],
    'Unit': ['%', 'ms', '%', '%', 'ms', 'Score'],
    'Status': ['Good', 'Warning', 'Good', 'Good', 'Warning', 'Good']
})

it_kpis['Achievement_Pct'] = (it_kpis['Current_Value'] / it_kpis['Target_Value'] * 100).round(1)
it_kpis['Progress_Pct'] = ((it_kpis['Current_Value'] - it_kpis['Min_Value']) / 
                          (it_kpis['Max_Value'] - it_kpis['Min_Value']) * 100).round(1)

# Special handling for response time and network latency (lower is better)
for metric in ['Response Time', 'Network Latency']:
    idx = it_kpis[it_kpis['KPI'] == metric].index[0]
    it_kpis.loc[idx, 'Achievement_Pct'] = (
        (it_kpis.loc[idx, 'Target_Value'] / it_kpis.loc[idx, 'Current_Value']) * 100).round(1)
    it_kpis.loc[idx, 'Progress_Pct'] = (
        (1 - (it_kpis.loc[idx, 'Current_Value'] - it_kpis.loc[idx, 'Min_Value']) / 
         (it_kpis.loc[idx, 'Max_Value'] - it_kpis.loc[idx, 'Min_Value'])) * 100).round(1)

# 4. Financial Health Dashboard
financial_kpis = pd.DataFrame({
    'KPI': ['Revenue Growth', 'Profit Margin', 'Cash Flow', 'Debt Ratio', 
            'ROI', 'Customer Acquisition Cost'],
    'Current_Value': [8.5, 18.2, 2.1, 35.0, 15.8, 85],
    'Target_Value': [10.0, 20.0, 2.5, 30.0, 18.0, 80],
    'Min_Value': [0, 0, -1.0, 0, 0, 50],
    'Max_Value': [25.0, 40.0, 5.0, 80.0, 30.0, 150],
    'Unit': ['%', '%', '$M', '%', '%', '$'],
    'Status': ['Good', 'Good', 'Good', 'Warning', 'Good', 'Warning']
})

financial_kpis['Achievement_Pct'] = (financial_kpis['Current_Value'] / financial_kpis['Target_Value'] * 100).round(1)
financial_kpis['Progress_Pct'] = ((financial_kpis['Current_Value'] - financial_kpis['Min_Value']) / 
                                 (financial_kpis['Max_Value'] - financial_kpis['Min_Value']) * 100).round(1)

# Special handling for debt ratio and CAC (lower is better)
for metric in ['Debt Ratio', 'Customer Acquisition Cost']:
    idx = financial_kpis[financial_kpis['KPI'] == metric].index[0]
    financial_kpis.loc[idx, 'Achievement_Pct'] = (
        (financial_kpis.loc[idx, 'Target_Value'] / financial_kpis.loc[idx, 'Current_Value']) * 100).round(1)
    financial_kpis.loc[idx, 'Progress_Pct'] = (
        (1 - (financial_kpis.loc[idx, 'Current_Value'] - financial_kpis.loc[idx, 'Min_Value']) / 
         (financial_kpis.loc[idx, 'Max_Value'] - financial_kpis.loc[idx, 'Min_Value'])) * 100).round(1)

# 5. Employee Satisfaction Survey Results
employee_kpis = pd.DataFrame({
    'KPI': ['Job Satisfaction', 'Work-Life Balance', 'Career Development', 
            'Management Quality', 'Compensation Satisfaction', 'Workplace Culture'],
    'Current_Value': [7.2, 6.8, 6.5, 7.8, 6.9, 8.1],
    'Target_Value': [8.0, 7.5, 7.0, 8.0, 7.5, 8.5],
    'Min_Value': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
    'Max_Value': [10.0, 10.0, 10.0, 10.0, 10.0, 10.0],
    'Unit': ['Score', 'Score', 'Score', 'Score', 'Score', 'Score'],
    'Status': ['Good', 'Warning', 'Warning', 'Good', 'Warning', 'Good']
})

employee_kpis['Achievement_Pct'] = (employee_kpis['Current_Value'] / employee_kpis['Target_Value'] * 100).round(1)
employee_kpis['Progress_Pct'] = ((employee_kpis['Current_Value'] - employee_kpis['Min_Value']) / 
                                (employee_kpis['Max_Value'] - employee_kpis['Min_Value']) * 100).round(1)

print("Sample gauge chart datasets created:")
print(f"Sales KPIs: {len(sales_kpis)} metrics")
print(f"Manufacturing KPIs: {len(manufacturing_kpis)} metrics")
print(f"IT Performance KPIs: {len(it_kpis)} metrics")
print(f"Financial KPIs: {len(financial_kpis)} metrics")
print(f"Employee Satisfaction: {len(employee_kpis)} metrics")

# Display sample data
print(f"\nSample Sales KPIs:")
print(sales_kpis[['KPI', 'Current_Value', 'Target_Value', 'Achievement_Pct', 'Unit']].head(3))


In [ ]:
# Function to create gauge charts
def create_gauge_chart(value, target, min_val, max_val, title, unit, ax, 
                      color_scheme='RdYlGn', show_target=True):
    """Create a gauge chart"""
    
    # Calculate angles (gauge spans 180 degrees)
    start_angle = 180
    end_angle = 0
    
    # Calculate value angle
    value_normalized = (value - min_val) / (max_val - min_val)
    value_angle = start_angle - (value_normalized * 180)
    
    # Calculate target angle
    target_normalized = (target - min_val) / (max_val - min_val)
    target_angle = start_angle - (target_normalized * 180)
    
    # Define color zones
    zones = [
        (0, 0.5, 'red'),      # Poor (0-50%)
        (0.5, 0.8, 'orange'), # Warning (50-80%)
        (0.8, 1.0, 'green')   # Good (80-100%)
    ]
    
    # Draw gauge background zones
    for zone_start, zone_end, color in zones:
        zone_start_angle = start_angle - (zone_start * 180)
        zone_end_angle = start_angle - (zone_end * 180)
        
        wedge = Wedge((0, 0), 1, zone_end_angle, zone_start_angle, 
                     facecolor=color, alpha=0.3, edgecolor='white', linewidth=2)
        ax.add_patch(wedge)
    
    # Draw gauge outline
    outline = Wedge((0, 0), 1, end_angle, start_angle, 
                   facecolor='none', edgecolor='black', linewidth=3)
    ax.add_patch(outline)
    
    # Draw inner circle (to create donut effect)
    inner_circle = Circle((0, 0), 0.6, facecolor='white', edgecolor='black', linewidth=2)
    ax.add_patch(inner_circle)
    
    # Draw needle
    needle_length = 0.8
    needle_x = needle_length * np.cos(np.radians(value_angle))
    needle_y = needle_length * np.sin(np.radians(value_angle))
    
    ax.plot([0, needle_x], [0, needle_y], color='black', linewidth=4, zorder=3)
    
    # Draw needle center
    center_circle = Circle((0, 0), 0.05, facecolor='black', zorder=4)
    ax.add_patch(center_circle)
    
    # Draw target indicator
    if show_target:
        target_x = 0.9 * np.cos(np.radians(target_angle))
        target_y = 0.9 * np.sin(np.radians(target_angle))
        ax.plot([0.8 * np.cos(np.radians(target_angle)), 1.0 * np.cos(np.radians(target_angle))], 
               [0.8 * np.sin(np.radians(target_angle)), 1.0 * np.sin(np.radians(target_angle))], 
               color='blue', linewidth=6, alpha=0.8, zorder=2)
        ax.text(target_x, target_y + 0.15, 'Target', ha='center', va='center', 
               fontsize=8, color='blue', fontweight='bold')
    
    # Add scale labels
    scale_positions = [0, 0.25, 0.5, 0.75, 1.0]
    for pos in scale_positions:
        angle = start_angle - (pos * 180)
        x = 1.1 * np.cos(np.radians(angle))
        y = 1.1 * np.sin(np.radians(angle))
        scale_value = min_val + pos * (max_val - min_val)
        ax.text(x, y, f'{scale_value:.1f}', ha='center', va='center', fontsize=8)
    
    # Add title and value display
    ax.text(0, -0.3, title, ha='center', va='center', fontsize=10, fontweight='bold')
    ax.text(0, -0.45, f'{value:.1f} {unit}', ha='center', va='center', 
           fontsize=12, fontweight='bold', color='black')
    
    # Calculate achievement percentage
    achievement_pct = (value / target * 100) if target > 0 else 0
    color = 'green' if achievement_pct >= 80 else 'orange' if achievement_pct >= 50 else 'red'
    ax.text(0, -0.6, f'{achievement_pct:.1f}% of target', ha='center', va='center', 
           fontsize=10, color=color, fontweight='bold')
    
    # Set equal aspect ratio and hide axes
    ax.set_xlim(-1.3, 1.3)
    ax.set_ylim(-0.8, 1.3)
    ax.set_aspect('equal')
    ax.axis('off')
    
    return ax

# Create basic gauge charts
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Gauge Chart Visualizations - KPI Dashboard', fontsize=16, fontweight='bold')

# Sales KPIs
sales_sample = sales_kpis.head(6)
for i, (idx, row) in enumerate(sales_sample.iterrows()):
    ax = axes[i//3, i%3]
    create_gauge_chart(row['Current_Value'], row['Target_Value'], 
                      row['Min_Value'], row['Max_Value'], 
                      row['KPI'], row['Unit'], ax)

plt.tight_layout()
plt.show()


In [ ]:
# Advanced gauge chart techniques
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Advanced Gauge Chart Techniques', fontsize=16, fontweight='bold')

# 1. Multi-needle gauge (comparing current vs previous period)
ax1 = axes[0, 0]

# Simulate previous period data
current_revenue = 2.8
previous_revenue = 2.5
target_revenue = 3.0
min_revenue = 0
max_revenue = 4.0

# Draw gauge background
zones = [(0, 0.5, 'red'), (0.5, 0.8, 'orange'), (0.8, 1.0, 'green')]
for zone_start, zone_end, color in zones:
    zone_start_angle = 180 - (zone_start * 180)
    zone_end_angle = 180 - (zone_end * 180)
    wedge = Wedge((0, 0), 1, zone_end_angle, zone_start_angle, 
                 facecolor=color, alpha=0.3, edgecolor='white', linewidth=2)
    ax1.add_patch(wedge)

# Draw gauge outline
outline = Wedge((0, 0), 1, 0, 180, facecolor='none', edgecolor='black', linewidth=3)
ax1.add_patch(outline)

# Inner circle
inner_circle = Circle((0, 0), 0.6, facecolor='white', edgecolor='black', linewidth=2)
ax1.add_patch(inner_circle)

# Current value needle (black)
current_normalized = (current_revenue - min_revenue) / (max_revenue - min_revenue)
current_angle = 180 - (current_normalized * 180)
current_x = 0.8 * np.cos(np.radians(current_angle))
current_y = 0.8 * np.sin(np.radians(current_angle))
ax1.plot([0, current_x], [0, current_y], color='black', linewidth=4, label='Current', zorder=3)

# Previous value needle (gray)
previous_normalized = (previous_revenue - min_revenue) / (max_revenue - min_revenue)
previous_angle = 180 - (previous_normalized * 180)
previous_x = 0.7 * np.cos(np.radians(previous_angle))
previous_y = 0.7 * np.sin(np.radians(previous_angle))
ax1.plot([0, previous_x], [0, previous_y], color='gray', linewidth=3, alpha=0.7, label='Previous', zorder=2)

# Center circle
center_circle = Circle((0, 0), 0.05, facecolor='black', zorder=4)
ax1.add_patch(center_circle)

# Labels and title
ax1.text(0, -0.3, 'Revenue Comparison', ha='center', va='center', fontsize=10, fontweight='bold')
ax1.text(0, -0.45, f'Current: ${current_revenue:.1f}M', ha='center', va='center', fontsize=10, color='black')
ax1.text(0, -0.55, f'Previous: ${previous_revenue:.1f}M', ha='center', va='center', fontsize=10, color='gray')

# Change indicator
change = ((current_revenue - previous_revenue) / previous_revenue) * 100
change_color = 'green' if change > 0 else 'red'
ax1.text(0, -0.65, f'{change:+.1f}% change', ha='center', va='center', 
         fontsize=10, color=change_color, fontweight='bold')

ax1.set_xlim(-1.3, 1.3)
ax1.set_ylim(-0.8, 1.3)
ax1.set_aspect('equal')
ax1.axis('off')

# 2. Horizontal progress bar gauge
ax2 = axes[0, 1]

# Manufacturing KPI example
oee_current = 82.5
oee_target = 85.0
oee_min = 0
oee_max = 100

# Draw horizontal gauge
gauge_width = 1.5
gauge_height = 0.3

# Background
background = Rectangle((-gauge_width/2, -gauge_height/2), gauge_width, gauge_height, 
                      facecolor='lightgray', edgecolor='black', linewidth=2)
ax2.add_patch(background)

# Progress fill
progress_width = (oee_current / oee_max) * gauge_width
progress_color = 'green' if oee_current >= oee_target else 'orange' if oee_current >= 60 else 'red'
progress = Rectangle((-gauge_width/2, -gauge_height/2), progress_width, gauge_height, 
                    facecolor=progress_color, alpha=0.7)
ax2.add_patch(progress)

# Target line
target_pos = (oee_target / oee_max) * gauge_width - gauge_width/2
ax2.plot([target_pos, target_pos], [-gauge_height/2 - 0.1, gauge_height/2 + 0.1], 
         color='blue', linewidth=4, label='Target')

# Value text
ax2.text(0, 0, f'{oee_current:.1f}%', ha='center', va='center', 
         fontsize=12, fontweight='bold', color='white')

# Scale markers
for value in [0, 25, 50, 75, 100]:
    x_pos = (value / oee_max) * gauge_width - gauge_width/2
    ax2.text(x_pos, -gauge_height/2 - 0.15, f'{value}', ha='center', va='top', fontsize=8)

ax2.text(0, 0.6, 'Overall Equipment Effectiveness', ha='center', va='center', 
         fontsize=10, fontweight='bold')
ax2.text(target_pos, gauge_height/2 + 0.2, f'Target: {oee_target}%', ha='center', va='bottom', 
         fontsize=9, color='blue', fontweight='bold')

ax2.set_xlim(-1, 1)
ax2.set_ylim(-0.5, 0.8)
ax2.set_aspect('equal')
ax2.axis('off')

# 3. Speedometer-style gauge with zones
ax3 = axes[1, 0]

# IT Performance - Response Time (lower is better)
response_time = 150
response_target = 100
response_min = 50
response_max = 500

# Calculate normalized values (inverted for "lower is better")
response_normalized = 1 - (response_time - response_min) / (response_max - response_min)
target_normalized = 1 - (response_target - response_min) / (response_max - response_min)

# Draw speedometer (270 degrees)
start_angle = 225
end_angle = -45

# Color zones (for performance metrics)
perf_zones = [
    (0, 0.4, 'red'),      # Poor performance
    (0.4, 0.7, 'orange'), # Warning
    (0.7, 1.0, 'green')   # Good performance
]

for zone_start, zone_end, color in perf_zones:
    zone_start_angle = start_angle - (zone_start * 270)
    zone_end_angle = start_angle - (zone_end * 270)
    wedge = Wedge((0, 0), 1, zone_end_angle, zone_start_angle, 
                 facecolor=color, alpha=0.3, edgecolor='white', linewidth=2)
    ax3.add_patch(wedge)

# Gauge outline
outline = Wedge((0, 0), 1, end_angle, start_angle, facecolor='none', edgecolor='black', linewidth=3)
ax3.add_patch(outline)

# Inner circle
inner_circle = Circle((0, 0), 0.6, facecolor='white', edgecolor='black', linewidth=2)
ax3.add_patch(inner_circle)

# Needle
needle_angle = start_angle - (response_normalized * 270)
needle_x = 0.8 * np.cos(np.radians(needle_angle))
needle_y = 0.8 * np.sin(np.radians(needle_angle))
ax3.plot([0, needle_x], [0, needle_y], color='black', linewidth=4, zorder=3)

# Center
center_circle = Circle((0, 0), 0.05, facecolor='black', zorder=4)
ax3.add_patch(center_circle)

# Scale labels (showing actual response time values)
scale_positions = [0, 0.25, 0.5, 0.75, 1.0]
for pos in scale_positions:
    angle = start_angle - (pos * 270)
    x = 1.1 * np.cos(np.radians(angle))
    y = 1.1 * np.sin(np.radians(angle))
    # Invert scale for "lower is better"
    scale_value = response_max - pos * (response_max - response_min)
    ax3.text(x, y, f'{scale_value:.0f}', ha='center', va='center', fontsize=8)

ax3.text(0, -0.2, 'Response Time', ha='center', va='center', fontsize=10, fontweight='bold')
ax3.text(0, -0.35, f'{response_time:.0f} ms', ha='center', va='center', 
         fontsize=12, fontweight='bold')

# Performance indicator
performance = "Good" if response_time <= response_target else "Warning" if response_time <= 200 else "Poor"
perf_color = 'green' if performance == "Good" else 'orange' if performance == "Warning" else 'red'
ax3.text(0, -0.5, performance, ha='center', va='center', 
         fontsize=10, color=perf_color, fontweight='bold')

ax3.set_xlim(-1.3, 1.3)
ax3.set_ylim(-1.3, 1.3)
ax3.set_aspect('equal')
ax3.axis('off')

# 4. Bullet chart (linear gauge)
ax4 = axes[1, 1]

# Financial KPI - ROI
roi_current = 15.8
roi_target = 18.0
roi_poor = 10.0
roi_good = 15.0
roi_excellent = 20.0
roi_max = 25.0

# Draw bullet chart
chart_width = 2.0
chart_height = 0.4

# Performance ranges
ranges = [
    (0, roi_poor/roi_max, 'red', 'Poor'),
    (roi_poor/roi_max, roi_good/roi_max, 'orange', 'Fair'),
    (roi_good/roi_max, roi_excellent/roi_max, 'lightgreen', 'Good'),
    (roi_excellent/roi_max, 1.0, 'green', 'Excellent')
]

y_center = 0
for start, end, color, label in ranges:
    width = (end - start) * chart_width
    x_start = start * chart_width - chart_width/2
    rect = Rectangle((x_start, y_center - chart_height/2), width, chart_height, 
                    facecolor=color, alpha=0.5, edgecolor='white', linewidth=1)
    ax4.add_patch(rect)

# Current value bar
current_width = (roi_current / roi_max) * chart_width
current_bar = Rectangle((-chart_width/2, y_center - chart_height/4), current_width, chart_height/2, 
                       facecolor='black', alpha=0.8)
ax4.add_patch(current_bar)

# Target marker
target_pos = (roi_target / roi_max) * chart_width - chart_width/2
ax4.plot([target_pos, target_pos], [y_center - chart_height/2 - 0.1, y_center + chart_height/2 + 0.1], 
         color='blue', linewidth=4, label='Target')

# Labels
ax4.text(0, y_center + chart_height/2 + 0.3, 'Return on Investment', ha='center', va='center', 
         fontsize=10, fontweight='bold')
ax4.text(-chart_width/2 - 0.1, y_center, f'{roi_current:.1f}%', ha='right', va='center', 
         fontsize=12, fontweight='bold')
ax4.text(target_pos, y_center + chart_height/2 + 0.15, f'Target: {roi_target:.1f}%', 
         ha='center', va='bottom', fontsize=9, color='blue')

# Scale
for value in [0, 5, 10, 15, 20, 25]:
    x_pos = (value / roi_max) * chart_width - chart_width/2
    ax4.text(x_pos, y_center - chart_height/2 - 0.15, f'{value}%', ha='center', va='top', fontsize=8)

ax4.set_xlim(-1.5, 1.5)
ax4.set_ylim(-0.6, 0.6)
ax4.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
# Interactive gauge charts (Plotly structure)
print("Interactive Gauge Charts (Plotly):")
print("=" * 50)

print("\n1. Basic Interactive Gauge")
print("Code structure:")
print("""
fig = go.Figure(go.Indicator(
    mode = "gauge+number+delta",
    value = current_value,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "KPI Title"},
    delta = {'reference': target_value},
    gauge = {
        'axis': {'range': [None, max_value]},
        'bar': {'color': "darkblue"},
        'steps': [
            {'range': [0, threshold1], 'color': "lightgray"},
            {'range': [threshold1, threshold2], 'color': "gray"}
        ],
        'threshold': {
            'line': {'color': "red", 'width': 4},
            'thickness': 0.75,
            'value': target_value
        }
    }
))

fig.update_layout(title="Interactive KPI Gauge")
fig.show()
""")

print("\n2. Multi-Gauge Dashboard")
print("Code structure:")
print("""
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=2, cols=3,
    specs=[[{'type': 'indicator'}, {'type': 'indicator'}, {'type': 'indicator'}],
           [{'type': 'indicator'}, {'type': 'indicator'}, {'type': 'indicator'}]],
    subplot_titles=('Revenue', 'Customers', 'Satisfaction', 'Market Share', 'Performance', 'Conversion')
)

# Add each gauge
for i, (kpi_name, current, target, max_val) in enumerate(kpi_data):
    row = (i // 3) + 1
    col = (i % 3) + 1
    
    fig.add_trace(go.Indicator(
        mode="gauge+number+delta",
        value=current,
        delta={'reference': target},
        title={'text': kpi_name},
        gauge={
            'axis': {'range': [None, max_val]},
            'bar': {'color': "darkblue"},
            'steps': [
                {'range': [0, max_val*0.5], 'color': "lightgray"},
                {'range': [max_val*0.5, max_val*0.8], 'color': "gray"}
            ],
            'threshold': {
                'line': {'color': "red", 'width': 4},
                'thickness': 0.75,
                'value': target
            }
        }
    ), row=row, col=col)

fig.update_layout(title="KPI Dashboard")
fig.show()
""")

print("\n3. Animated Gauge with Time Series")
print("Code structure:")
print("""
# Create animated gauge showing performance over time
fig = go.Figure()

# Add initial gauge
fig.add_trace(go.Indicator(
    mode="gauge+number+delta",
    value=initial_value,
    title={'text': "Performance Over Time"},
    gauge={'axis': {'range': [None, 100]}}
))

# Create animation frames
frames = []
for month, value in time_series_data:
    frame = go.Frame(
        data=[go.Indicator(
            mode="gauge+number+delta",
            value=value,
            title={'text': f"Performance - {month}"},
            gauge={'axis': {'range': [None, 100]}}
        )],
        name=month
    )
    frames.append(frame)

fig.frames = frames

# Add play controls
fig.update_layout(
    updatemenus=[
        dict(type="buttons",
             buttons=[dict(label="Play", method="animate", args=[None]),
                     dict(label="Pause", method="animate", args=[[None]])])
    ]
)

fig.show()
""")

print("\n4. Gauge with Custom Styling")
print("Code structure:")
print("""
fig = go.Figure(go.Indicator(
    mode = "gauge+number+delta",
    value = current_value,
    title = {'text': "Custom Styled Gauge", 'font': {'size': 24}},
    delta = {'reference': target_value, 'increasing': {'color': "green"}, 'decreasing': {'color': "red"}},
    number = {'font': {'size': 40}},
    gauge = {
        'axis': {'range': [None, max_value], 'tickwidth': 1, 'tickcolor': "darkblue"},
        'bar': {'color': "darkblue", 'thickness': 0.8},
        'bgcolor': "white",
        'borderwidth': 2,
        'bordercolor': "gray",
        'steps': [
            {'range': [0, 50], 'color': '#ffcccc'},
            {'range': [50, 80], 'color': '#ffffcc'},
            {'range': [80, 100], 'color': '#ccffcc'}
        ],
        'threshold': {
            'line': {'color': "red", 'width': 4},
            'thickness': 0.75,
            'value': target_value
        }
    }
))

fig.update_layout(
    paper_bgcolor="lavender",
    plot_bgcolor="white",
    font={'color': "darkblue", 'family': "Arial"}
)

fig.show()
""")


In [ ]:
# Dashboard layout with multiple gauge types
fig, axes = plt.subplots(3, 4, figsize=(20, 15))
fig.suptitle('Comprehensive KPI Dashboard - Multiple Gauge Types', fontsize=16, fontweight='bold')

# Function to create mini gauges for dashboard
def create_mini_gauge(value, target, min_val, max_val, title, unit, ax, gauge_type='circular'):
    """Create mini gauge for dashboard"""
    
    if gauge_type == 'circular':
        # Simplified circular gauge
        value_normalized = (value - min_val) / (max_val - min_val)
        value_angle = 180 - (value_normalized * 180)
        
        # Background arc
        arc = Wedge((0, 0), 1, 0, 180, facecolor='lightgray', alpha=0.3)
        ax.add_patch(arc)
        
        # Progress arc
        progress_angle = 180 - (value_normalized * 180)
        color = 'green' if value >= target else 'orange' if value >= min_val + (max_val - min_val) * 0.5 else 'red'
        progress_arc = Wedge((0, 0), 1, progress_angle, 180, facecolor=color, alpha=0.7)
        ax.add_patch(progress_arc)
        
        # Inner circle
        inner = Circle((0, 0), 0.7, facecolor='white')
        ax.add_patch(inner)
        
        # Value text
        ax.text(0, 0, f'{value:.1f}', ha='center', va='center', fontsize=12, fontweight='bold')
        ax.text(0, -0.3, unit, ha='center', va='center', fontsize=8)
        
    elif gauge_type == 'horizontal':
        # Horizontal progress bar
        bar_height = 0.3
        progress = (value - min_val) / (max_val - min_val)
        
        # Background
        bg = Rectangle((-1, -bar_height/2), 2, bar_height, facecolor='lightgray', alpha=0.3)
        ax.add_patch(bg)
        
        # Progress
        color = 'green' if value >= target else 'orange' if progress >= 0.5 else 'red'
        prog_bar = Rectangle((-1, -bar_height/2), 2 * progress, bar_height, facecolor=color, alpha=0.7)
        ax.add_patch(prog_bar)
        
        # Value text
        ax.text(0, 0, f'{value:.1f}{unit}', ha='center', va='center', fontsize=10, fontweight='bold')
    
    # Title
    ax.text(0, 0.8, title, ha='center', va='center', fontsize=8, fontweight='bold')
    
    # Target indicator
    target_pct = (value / target * 100) if target > 0 else 0
    target_color = 'green' if target_pct >= 90 else 'orange' if target_pct >= 70 else 'red'
    ax.text(0, -0.8, f'{target_pct:.0f}% of target', ha='center', va='center', 
           fontsize=7, color=target_color)
    
    ax.set_xlim(-1.2, 1.2)
    ax.set_ylim(-1, 1)
    ax.set_aspect('equal')
    ax.axis('off')

# Create dashboard with all KPI categories
dashboard_kpis = [
    # Sales metrics
    ('Revenue', 2.8, 3.0, 0, 4.0, '$M'),
    ('New Customers', 450, 500, 0, 600, ''),
    ('Satisfaction', 4.2, 4.5, 1, 5, '/5'),
    ('Market Share', 15.2, 18.0, 0, 25, '%'),
    
    # Operations metrics
    ('OEE', 82.5, 85.0, 0, 100, '%'),
    ('Quality', 96.2, 98.0, 0, 100, '%'),
    ('Safety', 8, 0, 0, 10, 'inc'),
    ('On-time', 94.5, 95.0, 0, 100, '%'),
    
    # IT metrics
    ('Uptime', 99.2, 99.9, 95, 100, '%'),
    ('Response', 150, 100, 50, 500, 'ms'),
    ('CPU', 68, 70, 0, 100, '%'),
    ('Security', 87, 90, 0, 100, 'pts')
]

gauge_types = ['circular'] * 4 + ['horizontal'] * 4 + ['circular'] * 4

for i, ((title, current, target, min_val, max_val, unit), gauge_type) in enumerate(zip(dashboard_kpis, gauge_types)):
    row = i // 4
    col = i % 4
    ax = axes[row, col]
    
    # Special handling for reverse metrics (lower is better)
    if title in ['Safety', 'Response']:
        # For safety incidents and response time, lower is better
        display_value = current
        # Adjust target achievement calculation
        if title == 'Safety':
            target_pct = (target / max(current, 0.1) * 100) if current > 0 else 100
            current_norm = 1 - (current / max_val)  # Invert for display
        else:  # Response time
            target_pct = (target / current * 100) if current > 0 else 0
            current_norm = 1 - ((current - min_val) / (max_val - min_val))
        
        create_mini_gauge(current_norm * max_val, target, min_val, max_val, title, unit, ax, gauge_type)
        
        # Override the value text for these metrics
        ax.texts[0].set_text(f'{display_value:.0f}')
        target_color = 'green' if target_pct >= 90 else 'orange' if target_pct >= 70 else 'red'
        if len(ax.texts) > 2:
            ax.texts[2].set_text(f'{target_pct:.0f}% of target')
            ax.texts[2].set_color(target_color)
    else:
        create_mini_gauge(current, target, min_val, max_val, title, unit, ax, gauge_type)

plt.tight_layout()
plt.show()


In [ ]:
# Statistical analysis of gauge chart data
print("Gauge Chart Statistical Analysis:")
print("=" * 50)

# 1. Performance Analysis Across Categories
print("1. KPI PERFORMANCE ANALYSIS:")

all_kpis = {
    'Sales': sales_kpis,
    'Manufacturing': manufacturing_kpis,
    'IT Performance': it_kpis,
    'Financial': financial_kpis,
    'Employee Satisfaction': employee_kpis
}

category_performance = []

for category, kpi_data in all_kpis.items():
    # Calculate category-level statistics
    avg_achievement = kpi_data['Achievement_Pct'].mean()
    metrics_on_target = len(kpi_data[kpi_data['Achievement_Pct'] >= 90])
    metrics_warning = len(kpi_data[(kpi_data['Achievement_Pct'] >= 70) & (kpi_data['Achievement_Pct'] < 90)])
    metrics_poor = len(kpi_data[kpi_data['Achievement_Pct'] < 70])
    
    category_performance.append({
        'Category': category,
        'Avg_Achievement': avg_achievement,
        'On_Target': metrics_on_target,
        'Warning': metrics_warning,
        'Poor': metrics_poor,
        'Total_Metrics': len(kpi_data)
    })
    
    print(f"\n   {category}:")
    print(f"     Average Achievement: {avg_achievement:.1f}%")
    print(f"     On Target (≥90%): {metrics_on_target}/{len(kpi_data)} metrics")
    print(f"     Warning (70-89%): {metrics_warning}/{len(kpi_data)} metrics")
    print(f"     Poor (<70%): {metrics_poor}/{len(kpi_data)} metrics")

# Overall performance ranking
performance_df = pd.DataFrame(category_performance)
performance_df = performance_df.sort_values('Avg_Achievement', ascending=False)

print(f"\n   Category Performance Ranking:")
for i, row in performance_df.iterrows():
    performance_status = "Excellent" if row['Avg_Achievement'] >= 90 else "Good" if row['Avg_Achievement'] >= 80 else "Needs Improvement"
    print(f"     {i+1}. {row['Category']}: {row['Avg_Achievement']:.1f}% ({performance_status})")

# 2. Target Achievement Analysis
print(f"\n2. TARGET ACHIEVEMENT PATTERNS:")

# Combine all KPIs for analysis
all_kpis_combined = pd.concat(all_kpis.values(), ignore_index=True)

# Achievement distribution
achievement_ranges = [
    (0, 50, 'Critical'),
    (50, 70, 'Poor'),
    (70, 80, 'Fair'),
    (80, 90, 'Good'),
    (90, 110, 'Excellent'),
    (110, float('inf'), 'Outstanding')
]

print("   Achievement Distribution:")
for min_val, max_val, status in achievement_ranges:
    if max_val == float('inf'):
        count = len(all_kpis_combined[all_kpis_combined['Achievement_Pct'] >= min_val])
        condition = f"≥{min_val}%"
    else:
        count = len(all_kpis_combined[
            (all_kpis_combined['Achievement_Pct'] >= min_val) & 
            (all_kpis_combined['Achievement_Pct'] < max_val)
        ])
        condition = f"{min_val}-{max_val}%"
    
    percentage = (count / len(all_kpis_combined)) * 100
    print(f"     {status} ({condition}): {count} KPIs ({percentage:.1f}%)")

# 3. Variance Analysis
print(f"\n3. PERFORMANCE VARIANCE ANALYSIS:")

for category, kpi_data in all_kpis.items():
    # Calculate variance in achievement rates
    achievement_std = kpi_data['Achievement_Pct'].std()
    achievement_range = kpi_data['Achievement_Pct'].max() - kpi_data['Achievement_Pct'].min()
    
    # Identify best and worst performing KPIs
    best_kpi = kpi_data.loc[kpi_data['Achievement_Pct'].idxmax()]
    worst_kpi = kpi_data.loc[kpi_data['Achievement_Pct'].idxmin()]
    
    consistency = "High" if achievement_std < 10 else "Moderate" if achievement_std < 20 else "Low"
    
    print(f"\n   {category}:")
    print(f"     Performance Consistency: {consistency} (std: {achievement_std:.1f}%)")
    print(f"     Performance Range: {achievement_range:.1f} percentage points")
    print(f"     Best KPI: {best_kpi['KPI']} ({best_kpi['Achievement_Pct']:.1f}%)")
    print(f"     Worst KPI: {worst_kpi['KPI']} ({worst_kpi['Achievement_Pct']:.1f}%)")

# 4. Target Setting Analysis
print(f"\n4. TARGET SETTING EVALUATION:")

print("   Target Realism Assessment:")
for category, kpi_data in all_kpis.items():
    # Calculate how ambitious targets are
    target_stretch = ((kpi_data['Target_Value'] - kpi_data['Current_Value']) / kpi_data['Current_Value'] * 100).mean()
    achievable_targets = len(kpi_data[kpi_data['Achievement_Pct'] >= 80])
    
    target_difficulty = "Ambitious" if target_stretch > 15 else "Moderate" if target_stretch > 5 else "Conservative"
    
    print(f"     {category}: {target_difficulty} targets (avg {target_stretch:.1f}% stretch)")
    print(f"       {achievable_targets}/{len(kpi_data)} targets within reach (≥80%)")

# 5. Seasonal and Trend Analysis (simulated)
print(f"\n5. PERFORMANCE TREND SIMULATION:")

# Simulate monthly performance data
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

print("   Simulated Monthly Performance Trends:")
for category in ['Sales', 'Manufacturing', 'IT Performance']:
    # Simulate seasonal patterns
    if category == 'Sales':
        # Sales typically peak in Q4
        seasonal_pattern = [0.85, 0.82, 0.88, 0.90, 0.92, 0.94, 0.89, 0.87, 0.91, 0.95, 0.98, 1.05]
    elif category == 'Manufacturing':
        # Manufacturing may have summer dips
        seasonal_pattern = [0.92, 0.94, 0.96, 0.93, 0.89, 0.85, 0.82, 0.84, 0.91, 0.95, 0.97, 0.93]
    else:  # IT Performance
        # IT performance may be more stable
        seasonal_pattern = [0.94, 0.95, 0.93, 0.96, 0.97, 0.95, 0.94, 0.93, 0.96, 0.98, 0.96, 0.95]
    
    avg_performance = np.mean(seasonal_pattern) * 100
    seasonal_volatility = np.std(seasonal_pattern) * 100
    
    print(f"     {category}: {avg_performance:.1f}% avg, {seasonal_volatility:.1f}% volatility")
    
    # Identify best and worst months
    best_month_idx = np.argmax(seasonal_pattern)
    worst_month_idx = np.argmin(seasonal_pattern)
    print(f"       Peak: {months[best_month_idx]} ({seasonal_pattern[best_month_idx]*100:.1f}%)")
    print(f"       Trough: {months[worst_month_idx]} ({seasonal_pattern[worst_month_idx]*100:.1f}%)")

# 6. Gauge Chart Design Effectiveness
print(f"\n6. GAUGE CHART DESIGN GUIDELINES:")

print("   Color Coding Best Practices:")
print("   ✓ Red (0-50%): Critical performance, immediate action needed")
print("   ✓ Orange (50-80%): Warning zone, requires attention")
print("   ✓ Green (80-100%): Good performance, on track")
print("   ✓ Blue indicators: Targets and benchmarks")
print("   ✓ Consistent color scheme across all gauges")

print(f"\n   Gauge Type Selection:")
print("   ✓ Circular gauges: Single KPI focus, executive dashboards")
print("   ✓ Horizontal bars: Space-efficient, multiple KPIs")
print("   ✓ Speedometer: Performance metrics, real-time monitoring")
print("   ✓ Bullet charts: Comparative analysis, detailed context")

print(f"\n   Scale and Range Considerations:")
print("   ✓ Use meaningful zero points")
print("   ✓ Set appropriate scale ranges (avoid cramping)")
print("   ✓ Include target indicators clearly")
print("   ✓ Show performance zones (poor/warning/good)")
print("   ✓ Consider logarithmic scales for wide ranges")

print(f"\n   Dashboard Layout Principles:")
print("   ✓ Most critical KPIs in upper-left position")
print("   ✓ Group related metrics together")
print("   ✓ Consistent gauge sizes within categories")
print("   ✓ Adequate white space between gauges")
print("   ✓ Clear titles and units")
print("   ✓ Update frequency indicators")

print(f"\nWhen to Use Gauge Charts:")
print("   • Executive dashboards and scorecards")
print("   • Real-time performance monitoring")
print("   • Goal tracking and progress visualization")
print("   • KPI reporting and status communication")
print("   • Performance against targets/benchmarks")
print("   • Quality control and operational metrics")

print(f"\nAlternatives to Consider:")
print("   • Progress bars for simple percentage completion")
print("   • Bullet charts for detailed performance context")
print("   • Sparklines for trend indication")
print("   • Traffic light indicators for simple status")
print("   • Bar charts for multiple KPI comparison")
print("   • Line charts for performance trends over time")
